In [23]:

train_msg = open("../../../MCMD/python/sort_random_train80_valid10_test10/train.msg.txt", encoding='UTF-8').read().split("\n")
len(train_msg)

In [3]:

test_diff = open("../../../MCMD/python/sort_random_train80_valid10_test10/test.diff.txt", encoding='UTF-8').read().split("\n")
len(test_diff)

In [4]:
test_diff[0]

In [7]:
valid_msg = open("../../MCMD/java/sort_random_train80_valid10_test10/valid.msg.txt", encoding='UTF-8').read().split("\n")
len(valid_msg)

In [9]:
test_msg = open("../../MCMD/java/sort_random_train80_valid10_test10/test.msg.txt", encoding='UTF-8').read().split("\n")
len(test_msg)

In [12]:

train_diff = open("../../../MCMD/python/sort_random_train80_valid10_test10/train.diff.txt", encoding='UTF-8').read().split("\n")


In [13]:
len(train_diff)

## Preprocess script

In [42]:
import pickle

# repo_raw_data = pickle.load(open("../../../MCMD/raw_data/javascript/angular/angular.js.pickle","rb"))

In [28]:
from tqdm import tqdm
import glob
import os
language = 'javascript'
folder_path = f'../../../MCMD/raw_data/{language}'
file_paths = glob.glob(os.path.join(folder_path, '**', '*'), recursive=True)
file_paths = [f for f in file_paths if os.path.isfile(f)]

In [43]:
target_data = []

for file in file_paths:
    repo_raw_data = pickle.load(open(file,"rb"))
    for data in tqdm(repo_raw_data, total=len(repo_raw_data)):
        if data['diff'] is None:
            continue
        if data['diff'].count('.js ')==1 and data['diff'].count('.js\n')==3 and data['diff'].count('diff --git')==1:
            item = {
                'msg': data['msg'],
                'diff': data['diff'],
                'date': data['date'],
                'repo': '/'.join(file.split('\\')[-2:]).split('.')[0]
            }
            target_data.append(item)

100%|██████████| 1077/1077 [00:00<00:00, 99015.07it/s]


In [45]:
for data in target_data:
    data['msg'] = data['msg'].lstrip()

In [1]:
import re

In [2]:
def check_angular_convention(msg, has_space=True, strict=True):
    if has_space:
        pattern = '((chore)|(docs)|(feat)|(fix)|(perf)|(refactor)|(style)|(test))\s(\((\s|\S)+\)\s)?:(\s(\s|\S)+)?'
    else:
        pattern = '((chore)|(docs)|(feat)|(fix)|(perf)|(refactor)|(style)|(test))(\((\s|\S)+\))?:(\s\S+(\s|\S)+)?'
    if not strict:
        pattern = '^\s*((chore)|(docs)|(feat)|(fix)|(perf)|(refactor)|(style)|(test))'
    return re.match(pattern, msg) != None

In [46]:
mcmd_strict_match_template_list = []
for index, item in enumerate(target_data):
    if check_angular_convention(item['msg'], has_space=False) or check_angular_convention(item['msg'], has_space=True):
        mcmd_strict_match_template_list.append(item)

In [50]:
# filter angular_filter_data by the length of the diff(1500) and msg(500)
mcmd_filtered_length_data = [item for item in mcmd_strict_match_template_list if len(item['diff'])<=500 and len(item['msg'])<=100]

In [1]:
import json
with open('../../llm4commit/data/final_preprocessed_data/discriminator/db_data_js.json', 'r', encoding='UTF-8') as f:
    target_data = json.load(f)

In [3]:
import re

def check_angular_convention(msg, has_space=True):
    types = '((build)|(ci)|(docs)|(feat)|(fix)|(perf)|(refactor)|(style)|(test)|(chore)|(revert))'
    if has_space:
        pattern = f'{types}\s(\((\s|\S)+\)\s)?:(\s(\s|\S)+)?'
    else:
        pattern = f'{types}(\((\s|\S)+\))?:(\s\S+(\s|\S)+)?'
    return re.match(pattern, msg) is not None


In [4]:
chronicle_strict_match_template_list = []
for index, item in enumerate(target_data):
    if check_angular_convention(item['msg'], has_space=False) or check_angular_convention(item['msg'], has_space=True):
        chronicle_strict_match_template_list.append(item)

In [5]:
len(chronicle_strict_match_template_list)

15963

In [55]:
def find_common_msg_indices(list1, list2):
    common_indices = []

    for index1, dict1 in enumerate(list1):
        for index2, dict2 in enumerate(list2):
            if dict1.get('msg') == dict2.get('msg'):
                common_indices.append((index1, index2))
    
    return common_indices

In [56]:
common_indices = find_common_msg_indices(mcmd_filtered_length_data, chronicle_strict_match_template_list)

In [57]:
len(common_indices)

148

In [60]:
import json
with open('../../llm4commit/data/angular_filtered/data_js.json', 'w', encoding='UTF-8') as f:
    json.dump(chronicle_strict_match_template_list, f, ensure_ascii=False, indent=4)

In [61]:
len(chronicle_strict_match_template_list)

18048

In [6]:
# shuffle the data, 13000 for rag db, for the rest, 4/5 for training, 1/10 for validation, 1/10 for testing
import random
from sklearn.model_selection import train_test_split

def split_data(data, db_size=11000, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # Step 1: Shuffle the data
    random.seed(42)
    random.shuffle(data)
    
    # Step 2: Split off the first 13000 elements for the rag db
    rag_db = data[:db_size]
    remaining_data = data[db_size:]
    
    # Step 3: Use train_test_split to split the remaining data
    train_data, temp_data = train_test_split(remaining_data, test_size=(val_ratio + test_ratio))
    val_data, test_data = train_test_split(temp_data, test_size=test_ratio / (val_ratio + test_ratio))
    
    return rag_db, train_data, val_data, test_data

In [7]:
rag_db, train_data, val_data, test_data = split_data(chronicle_strict_match_template_list)

In [9]:
with open('../../llm4commit/data/angular_filtered/data_js.json', 'w', encoding='UTF-8') as f:
    json.dump(chronicle_strict_match_template_list, f, ensure_ascii=False, indent=4)

In [8]:
# save the data
import json
with open('../../llm4commit/data/angular_filtered/rag_db_js.json', 'w', encoding='UTF-8') as f:
    json.dump(rag_db, f, ensure_ascii=False, indent=4)
with open('../../llm4commit/data/angular_filtered/train_data_js.json', 'w', encoding='UTF-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)
with open('../../llm4commit/data/angular_filtered/val_data_js.json', 'w', encoding='UTF-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=4)
with open('../../llm4commit/data/angular_filtered/test_data_js.json', 'w', encoding='UTF-8') as f:
    json.dump(test_data, f, ensure_ascii=False, indent=4)

In [23]:
target_data[1]['diff'].count('diff --git')

14

In [16]:
target_data[1020]['diff'].count('.js ')

1

In [30]:
import json
with open('../../../MCMD/raw_data/js_after_length_filter.json', 'w', encoding='UTF-8') as f:
    json.dump(target_data, f, ensure_ascii=False, indent=4)

In [82]:
len(target_data)

110717

In [14]:
target_data[65131]

{'msg': 'fix : Channel closed error in ProcessContainer',
 'diff': "diff --git a/lib/ProcessContainer.js b/lib/ProcessContainer.js\nindex c31fe35ba..ad624c76e 100644\n--- a/lib/ProcessContainer.js\n+++ b/lib/ProcessContainer.js\n@@ -33,6 +33,14 @@ delete process.env.pm2_env;\n   var script      = pm2_env.pm_exec_path;\n   var cronRestart = pm2_env.cron_restart;\n \n+  var original_send = process.send;\n+  process.send = function() {\n+    if (process.connected)\n+      original_send.apply(this, arguments);\n+    else\n+      console.error('IPC channel with master closed');\n+  };\n+\n   if (cst.MODIFY_REQUIRE)\n     require.main.filename = pm2_env.pm_exec_path;\n \n",
 'source': 'Unitech'}

In [13]:
len(target_data)

67230

In [49]:
## output all msg to a txt file
with open('../../../MCMD/raw_data/js_after_length_filter.msg.txt', 'w', encoding='UTF-8') as f:
    for item in target_data:
        f.write(item['msg'].replace('\n', '\\n').replace('\r', '\\r').lstrip() + '\n')

In [32]:
len(repo_raw_data[0]['diff'])

2523

In [30]:
len(repo_raw_data[0]['diff'].split(" "))

145

In [26]:
assert len(train_diff) == len(train_msg)

In [ ]:
def process_diff(diff_text):
    # Replace <nl> with \n
    diff_text = diff_text.replace('<nl>', '\n')
    diff_text = re.sub(r'(?<=\S)\s*([^\w\s])\s*(?=\S)', r'\1', diff_text)
    diff_text = re.sub(r'\n\s+', r'\n', diff_text)
    diff_text = diff_text.replace('mmm', 'diff --git')
    diff_text = diff_text.replace('\nppp', '')
    return diff_text

In [32]:
from tqdm import tqdm
diffs = []
msgs = []
for index, train_diff_item in tqdm(enumerate(train_diff), total=len(train_diff)):
    diff = process_diff(train_diff_item)
    msg = train_msg[index]
    if diff.count(".py")==2 and len(diff)<=300 and len(msg)<=50:
        diffs.append(diff)
        msgs.append(msg)

In [35]:
# Output the diffs list to a text file
output_diff_file = '../../../MCMD/python/sort_random_train80_valid10_test10/train_diff_processed.txt'
output_msg_file = '../../../MCMD/python/sort_random_train80_valid10_test10/train_msg_processed.txt'
with open(output_diff_file, 'w', encoding='UTF-8') as f:
    for diff in diffs:
        f.write(diff + '\n')
with open(output_msg_file, 'w', encoding='UTF-8') as f:
    for msg in msgs:
        f.write(msg + '\n')


In [11]:
print(len(diffs))

In [5]:
import json

selected_messages = []
selected_languages = []
selected_diffs = []
nmt_msg = []
nngen_msg = []

def extract_diffs(indices, diff_file_path, nngen_path, nmt_path):
    with open(diff_path, 'r', encoding='UTF-8') as diff_file:
        diff_line = diff_file.readlines()
    
    with open(nngen_path, 'r', encoding='UTF-8') as nngen_file:
        nngen_line = nngen_file.readlines()

    with open(nmt_path, 'r', encoding='UTF-8') as nmt_file:
        nmt_line = nmt_file.readlines()
    
    
    for i in indices:
        diff = process_diff(diff_line[i])
        nngen = nngen_line[i]
        nmt = nmt_line[i]
        selected_diffs.append(diff)
        nngen_msg.append(nngen)
        nmt_msg.append(nmt)

mcmd_base_path = "../../MCMD"
models_base_path = "./data"

languages = ["java", "cpp", "csharp", "python", "javascript"]
file_names = ["test.msg.txt", "test.diff.txt"]

test_msg_path = [f"{mcmd_base_path}/{lang}/sort_random_train80_valid10_test10/test.msg.txt" for lang in languages]
test_diff_path = [f"{mcmd_base_path}/{lang}/sort_random_train80_valid10_test10/test.diff.txt" for lang in languages]

nngen_msg_path = [f"{models_base_path}/model_NNGen/data_MCMD/{lang}/split_random/gen.msg" for lang in languages]
nmt_msg_path = [f"{models_base_path}/model_NMT/data_MCMD/{lang}/split_random/gen.msg" for lang in languages]


for msg_path, diff_path, nngen_path, nmt_path in zip(test_msg_path, test_diff_path, nngen_msg_path, nmt_msg_path):
    selected_indices = []
    with open(msg_path, 'r', encoding='UTF-8') as f:
        count = 0
        for i, line in enumerate(f):
            if count>= 1000:
                break
            # line = process_msg(line)
            if len(line.split()) >= 5:
                selected_indices.append(i)
                selected_messages.append(line.strip())
                selected_languages.append(msg_path.split('/')[3])
                count += 1
            
        extract_diffs(selected_indices, diff_path, nngen_path, nmt_path)

# Create a list of dictionaries containing selected messages and diffs
data = []
for msg, diff, nngen, nmt, language in zip(selected_messages, selected_diffs, nngen_msg, nmt_msg, selected_languages):
    item = {
        'msg': msg,
        'diff': diff,
        'nngen': nngen,
        'nmt': nmt,
        'language': language
    }
    data.append(item)

# Write the data to a JSON file
with open('data/selected_data.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


In [20]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base", cache_dir='./')
model = RobertaForCausalLM.from_pretrained("microsoft/codebert-base", cache_dir='./', is_decoder=True)
model.to(device)

In [23]:
def generate_description(code):
    inputs = tokenizer(code, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=128, num_return_sequences=1)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

In [25]:
generate_description("The following is a diff which describes the code changes in a commit, Your task is to write a short commit message accordingly. diff --git a/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java b/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java\nindex e8d986667..ce243ca1f 100644\n--- a/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java\n+++ b/src/gwt/src/org/rstudio/studio/client/workbench/views/source/editors/text/TextEditingTargetRenameHelper.java\n@@ -44,7 +44,7 @@ public class TextEditingTargetRenameHelper\n       \n       editor_.setCursorPosition(position);\n       \n-      // Validate that we're looking at an R identifier (TODO: refactor strings?)\n+      // Validate that we're looking at an R identifier\n       String targetValue = cursor.currentValue();\n       String targetType = cursor.currentType();\n       \n@@ -104,7 +104,6 @@ public class TextEditingTargetRenameHelper\n       }\n       \n       // Otherwise, just rename the variable within the current scope.\n-      // TODO: Do we need to look into parent scopes?\n       return renameVariablesInScope(\n             editor_.getCurrentScope(),\n             targetValue,\nAccording to the diff, the commit message should be:")